In [1]:
!which python

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/bin/python


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
# Model from Hugging Face hub
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# New instruction dataset
stack_overflow_dataset = "Mxode/StackOverflow-QA-C-Language-40k"
# Fine-tuned model
new_model = "llama-2-7b-chat-stack-overflow"

In [4]:
dataset = load_dataset(stack_overflow_dataset, split="train")

In [5]:
# dataset currently has 'question' and 'answer' columns. We need to combine them into a single column named 'text'
dataset = dataset.map(
    lambda examples: {
        "text": "<s>[INST]" + examples["question"] + "[\INST] " + examples["answer"] + "</s>",
    }
)

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

  0%|          | 0/10162 [00:00<?, ?it/s]

{'loss': 2.1549, 'grad_norm': 0.3336416184902191, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 2.4767, 'grad_norm': 1.453829288482666, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 2.002, 'grad_norm': 0.42404285073280334, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 2.2998, 'grad_norm': 0.7494381666183472, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.9278, 'grad_norm': 0.4462108910083771, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 2.1755, 'grad_norm': 0.6799669861793518, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.8452, 'grad_norm': 0.3923417031764984, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 2.1849, 'grad_norm': 0.6918219327926636, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.8184, 'grad_norm': 0.3401746451854706, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 2.1991, 'grad_norm': 0.8835895657539368, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.8687, 'grad_norm': 0.3712921142578125, 'learning_rate': 0.0002, 'epoch': 0.03}
{'loss': 2.15

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bd2a21df-0199-4317-881f-96b5bc64a68f)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 2.0908, 'grad_norm': 0.5399209260940552, 'learning_rate': 0.0002, 'epoch': 0.84}


/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x76e2b067f280>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: a4af9ad6-e647-4762-989e-954c3bb827da)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(


...

{'loss': 1.7599, 'grad_norm': 0.321511834859848, 'learning_rate': 0.0002, 'epoch': 0.86}


/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x76e2b072ef50>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: eac51428-83c7-43d8-b192-33bf75badfc2)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 2.1096, 'grad_norm': 0.7874270081520081, 'learning_rate': 0.0002, 'epoch': 0.87}


/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x76e2b067e950>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: dab2ee87-2303-4544-8e01-5a3b6b61766a)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.7635, 'grad_norm': 0.7014102339744568, 'learning_rate': 0.0002, 'epoch': 0.87}
{'loss': 2.1031, 'grad_norm': 0.8954417109489441, 'learning_rate': 0.0002, 'epoch': 0.87}
{'loss': 1.7226, 'grad_norm': 0.3867749869823456, 'learning_rate': 0.0002, 'epoch': 0.87}
{'loss': 2.0848, 'grad_norm': 0.9625787734985352, 'learning_rate': 0.0002, 'epoch': 0.88}
{'loss': 1.7205, 'grad_norm': 0.4066599905490875, 'learning_rate': 0.0002, 'epoch': 0.88}
{'loss': 2.018, 'grad_norm': 0.7131065130233765, 'learning_rate': 0.0002, 'epoch': 0.88}
{'loss': 1.7381, 'grad_norm': 0.3091999292373657, 'learning_rate': 0.0002, 'epoch': 0.88}
{'loss': 2.0922, 'grad_norm': 1.1560139656066895, 'learning_rate': 0.0002, 'epoch': 0.89}
{'loss': 1.7891, 'grad_norm': 0.3340603709220886, 'learning_rate': 0.0002, 'epoch': 0.89}
{'loss': 2.0563, 'grad_norm': 0.5417934656143188, 'learning_rate': 0.0002, 'epoch': 0.89}
{'loss': 1.7593, 'grad_norm': 0.3191429376602173, 'learning_rate': 0.0002, 'epoch': 0.89}
{'loss': 2.

In [14]:
trainer.push_to_hub("tiny-llama-stack-overflow")

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713400697.helios.16772.0:   0%|          | 0.00/91.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1713400569.helios.16587.0:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/36.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bgsmagnuson/results/commit/af3ab334a613607d0a63f574ff52a15b428f4b49', commit_message='tiny-llama-stack-overflow', commit_description='', oid='af3ab334a613607d0a63f574ff52a15b428f4b49', pr_url=None, pr_revision=None, pr_num=None)